# 生成对抗网络

在某种形式上，我们使用了深度神经网络学习的从数据点到标签的映射。 这种学习称为判别学习，例如，我们希望能够区分照片中的猫和狗中的照片。 分类器和回归器都是歧视性学习的例子。 通过反向传播训练的神经网络颠覆了我们认为关于大型复杂数据集的判别式学习的所有知识。 在短短5至6年间，高分辨率图像的分类精度已从无用变成了人类级别（有些警告）。 我们将不为您所困扰，因为深度神经网络的所有其他判别任务都表现出色。

2014年，一篇突破性的论文介绍了生成对抗网络（GANs）“ Goodfellow.Pouget-Abadie.Mirza.ea.2014”，这是一种利用判别模型的力量来获得良好生成模型的聪明新方法。 GAN的核心思想是，如果我们不能将假数据与真实数据区分开，那么数据生成器就很好。在统计中，这称为两次抽样检验-回答以下问题的检验：数据集 $ X = \ {x_1，\ ldots，x_n \} $ 和 $ X'= \ {x'_1，\ ldots，x' _n \} $ 是从同一分布中提取的。大多数统计文件与GAN之间的主要区别在于，后者以建设性的方式使用了这一思想。换句话说，他们不只是训练模型说“嘿，这两个数据集看起来不像它们来自同一分布”，而是使用了[两个样本检验]（https://en.wikipedia.org/ wiki / Two-sample_hypothesis_testing）为生成的模型提供训练信号。这使我们能够改进数据生成器，直到它生成类似于真实数据的内容为止。至少，它需要愚弄分类器。即使我们的分类器是最先进的深度神经网络。

![Image Name](https://cdn.kesci.com/upload/image/q5tv0m8ro4.jpg?imageView2/0/w/320/h/320)

## 生成器

我们的生成机网络将是最简单的网络-单层线性模型。 这是因为我们将使用高斯数据生成器来驱动线性网络。 因此，它实际上只需要学习参数就可以完美地伪造事物。

In [ ]:
class net_G(nn.Module):
    def __init__(self):
        super(net_G,self).__init__()
        self.model=nn.Sequential(
            nn.Linear(2,2),
        )
        self._initialize_weights()
    def forward(self,x):
        x=self.model(x)
        return x
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m,nn.Linear):
                m.weight.data.normal_(0,0.02)
                m.bias.data.zero_()

## 鉴别器

对于鉴别器，我们将更具区分性：我们将使用具有3层的MLP来使事情变得更有趣。

In [ ]:
class net_D(nn.Module):
    def __init__(self):
        super(net_D,self).__init__()
        self.model=nn.Sequential(
            nn.Linear(2,5),
            nn.Tanh(),
            nn.Linear(5,3),
            nn.Tanh(),
            nn.Linear(3,1),
            nn.Sigmoid()
        )
        self._initialize_weights()
    def forward(self,x):
        x=self.model(x)
        return x
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m,nn.Linear):
                m.weight.data.normal_(0,0.02)
                m.bias.data.zero_()

## 摘要

*生成对抗网络（GAN）由两个深层网络组成，即生成器和鉴别器。
*生成器通过最大化交叉熵损失*即$ \ max \ log（D（\（mathbf {x'}））$）来生成尽可能接近真实图像的图像，以欺骗鉴别器。
*鉴别器试图通过最小化交叉熵损失* ie *，$ \ min-y \ log D（\ mathbf {x}）-（1-y）\ log（ 1-D（\ mathbf {x}））$。

# 深度卷积生成对抗网络

我们介绍了GAN的工作原理。 我们证明了他们可以从一些简单，易于采样的分布（如均匀分布或正态分布）中抽取样本，并将它们转换为看起来与某些数据集的分布相匹配的样本。 尽管我们提出的匹配2D高斯分布的例子很明确，但这并不是特别令人兴奋。

在本节中，我们将演示如何使用GAN生成逼真的图像。 我们将基于在Radford.Metz.Chintala.2015中引用的深度卷积GAN（DCGAN）建立模型。 我们将借鉴已证明在区分计算机视觉问题上如此成功的卷积架构，并展示如何通过GAN来利用它们来生成逼真的图像。

## 生成器

生成器需要将噪声变量$ \ mathbf z \ in \ mathbb R ^ d $（长度-$ d $向量）映射到宽度和高度为$ 64 \乘以64 $的RGB图像。 在sec_fcn中，我们介绍了使用转置卷积层的完全卷积网络（请参阅sec_transposed_conv）来扩大输入大小。 生成器的基本块包含一个转置的卷积层，然后进行批量归一化和ReLU激活。

In [ ]:
class G_block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4,strides=2, padding=1):
        super(G_block,self).__init__()
        self.conv2d_trans=nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size,
                                             stride=strides, padding=padding, bias=False)
        self.batch_norm=nn.BatchNorm2d(out_channels,0.8)
        self.activation=nn.ReLU()
    def forward(self,x):
        return self.activation(self.batch_norm(self.conv2d_trans(x)))

生成器由四个基本块组成，这些块将输入的宽度和高度从1增加到32。同时，它首先将潜变量投影到$ 64 \ x 8 $通道中，然后每次将通道减半。 最后，转置的卷积层用于生成输出。 它将宽度和高度进一步加倍，以匹配所需的$ 64 \乘以64 $形状，并将通道大小减小为$ 3 $。 tanh激活函数适用于$（-1，1）$范围内的项目输出值。

In [ ]:
class net_G(nn.Module):
    def __init__(self,in_channels):
        super(net_G,self).__init__()

        n_G=64
        self.model=nn.Sequential(
            G_block(in_channels,n_G*8,strides=1,padding=0),
            G_block(n_G*8,n_G*4),
            G_block(n_G*4,n_G*2),
            G_block(n_G*2,n_G),
            nn.ConvTranspose2d(
                n_G,3,kernel_size=4,stride=2,padding=1,bias=False
            ),
            nn.Tanh()
        )
    def forward(self,x):
        x=self.model(x)
        return x


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, mean=0, std=0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, mean=1.0, std=0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

## 鉴别器

鉴别器是普通的卷积网络，只是它使用泄漏的ReLU作为其激活功能。 给定$ \ alpha \ in [0，1] $，其定义为


$$
\ textrm {leaky ReLU}（x）= \ begin {cases} x＆\ text {if} \ x> 0 \\ \ alpha x＆\ text {否则} \ end {cases}。
$$


In [ ]:
class D_block(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=4,strides=2,
                 padding=1,alpha=0.2):
        super(D_block,self).__init__()
        self.conv2d=nn.Conv2d(in_channels,out_channels,kernel_size,strides,padding,bias=False)
        self.batch_norm=nn.BatchNorm2d(out_channels,0.8)
        self.activation=nn.LeakyReLU(alpha)
    def forward(self,X):
        return self.activation(self.batch_norm(self.conv2d(X)))

In [ ]:
class net_D(nn.Module):
    def __init__(self,in_channels):
        super(net_D,self).__init__()
        n_D=64
        self.model=nn.Sequential(
            D_block(in_channels,n_D),
            D_block(n_D,n_D*2),
            D_block(n_D*2,n_D*4),
            D_block(n_D*4,n_D*8)
        )
        self.conv=nn.Conv2d(n_D*8,1,kernel_size=4,bias=False)
        self.activation=nn.Sigmoid()
        # self._initialize_weights()
    def forward(self,x):
        x=self.model(x)
        x=self.conv(x)
        x=self.activation(x)
        return x

## 小结


* DCGAN体系结构具有四个用于鉴别器的卷积层和四个用于生成器的“小跨度”卷积层。
*鉴别器是一个具有批归一化（除其输入层之外）和泄漏性ReLU激活的4层跨卷积。
* Leaky ReLU是一个非线性函数，为负输入提供非零输出。 它旨在解决“垂死的ReLU”问题，并帮助渐变在整个体系结构中更轻松地流动。